# Лабораторная работа 8
---
### Упражнение 8.1
> Что случится, если при увеличении ширины гауссова окна `std` не увеличивать число элементов в окне `M`

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from thinkdsp import decorate
from thinkdsp import SquareSignal
from thinkdsp import SawtoothSignal
from thinkdsp import Wave
import scipy.signal

def zero_pad(array, n):
    """Extends an array with zeros.

    array: NumPy array
    n: length of result

    returns: new NumPy array
    """
    res = np.zeros(n)
    res[:len(array)] = array
    return res

def plot_filter(M=11, std=2):
    signal = SquareSignal(freq=440)
    wave = signal.make_wave(duration=1, framerate=44100)
    spectrum = wave.make_spectrum()

    gaussian = scipy.signal.gaussian(M=M, std=std)
    gaussian /= sum(gaussian)

    ys = np.convolve(wave.ys, gaussian, mode='same')
    smooth =  Wave(ys, framerate=wave.framerate)
    spectrum2 = smooth.make_spectrum()

    # plot the ratio of the original and smoothed spectrum
    amps = spectrum.amps
    amps2 = spectrum2.amps
    ratio = amps2 / amps    
    ratio[amps<560] = 0

    # plot the same ratio along with the FFT of the window
    padded =  zero_pad(gaussian, len(wave))
    dft_gaussian = np.fft.rfft(padded)

    plt.plot(np.abs(dft_gaussian), color='gray', label='Gaussian filter')
    plt.plot(ratio, label='amplitude ratio')

    decorate(xlabel='Frequency (Hz)', ylabel='Amplitude ratio')
    plt.show()

Данная функция принимаем на вход количество элементов в окне и стандартное отклонение нормального распределения (распределение Гаусса), что также является шириной гауссова окна. Сначала создается прямоугольный сигнал и сглаживает его гауссовым окном. После чего отображает на графике окно после преобразования Фурье и результат применения окна

In [4]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

slider = widgets.IntSlider(min=2, max=100, value=11)
slider2 = widgets.FloatSlider(min=0, max=20, value=2)
interact(plot_filter, M=slider, std=slider2);

interactive(children=(IntSlider(value=11, description='M', min=2), FloatSlider(value=2.0, description='std', m…

Как мы можем заметить при уменьшении только ширины окна, оно начинает меньше влиять на частоты, спадая медленнее, а при увеличении наоборот. Также при увеличении `std` появляются боковые лепестки